# Crawling PTA TRUNOJOYO

In [1]:
!pip install builtwith
!pip install beautifulsoup4
!pip install pandas
!pip install tqdm
!pip install openpyxl




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

def get_fakultas_prodi_list():
    """
    Fungsi untuk mengambil daftar semua Fakultas dan Prodi di dalamnya.
    Struktur scraping disesuaikan dengan sidebar di mana prodi berada di dalam fakultas.
    """
    prodi_list = []
    try:
        # URL yang berisi sidebar navigasi fakultas dan prodi
        url_nav = "https://pta.trunojoyo.ac.id/c_search/byfac"
        r = requests.get(url_nav)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        
        # 1. Targetkan sidebar navigasi utama
        sidebar_nav = soup.select_one('div.box.sidebar_nav')
        if not sidebar_nav:
            print("Sidebar navigasi tidak ditemukan.")
            return []

        # 2. Ambil semua item list <li> dari fakultas (level pertama)
        fakultas_items = sidebar_nav.select_one('ul').find_all('li', recursive=False)
        
        for item_fakultas in fakultas_items:
            # 3. Ambil nama fakultas dari link <a> pertama di dalam <li>
            anchor_fakultas = item_fakultas.find('a', recursive=False)
            if not anchor_fakultas:
                continue
            nama_fakultas = anchor_fakultas.get_text(strip=True)
            
            # 4. Cari daftar <ul> bersarang yang berisi prodi
            ul_prodi = item_fakultas.find('ul')
            if not ul_prodi:
                continue

            # 5. Ambil semua link <a> prodi di dalam daftar bersarang tersebut
            for link_prodi in ul_prodi.select('li a'):
                nama_prodi = link_prodi.get_text(strip=True)
                href = link_prodi.get('href')
                # Ekstrak ID prodi dari URL
                prodi_id = href.strip('/').split('/')[-1]
                
                if prodi_id.isdigit():
                    prodi_list.append({
                        "id_prodi": int(prodi_id),
                        "nama_prodi": nama_prodi,
                        "nama_fakultas": nama_fakultas
                    })
                    
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar fakultas dan prodi: {e}")
        
    return prodi_list

def scrape_all():
    """
    Fungsi utama untuk melakukan scraping data dari semua prodi,
    kini dengan informasi fakultas.
    """
    # 1. Dapatkan daftar lengkap prodi beserta fakultasnya
    daftar_prodi_lengkap = get_fakultas_prodi_list()

    if not daftar_prodi_lengkap:
        print("Tidak ada prodi yang bisa di-scrape. Program berhenti.")
        return

    # Inisialisasi dictionary dengan kolom baru "nama_fakultas"
    data = {
        "nama_fakultas": [],
        "id_prodi": [],
        "nama_prodi": [],
        "penulis": [],
        "judul": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "abstrak": [],
        "abstrak_inggris": []
    }

    # 2. Lakukan looping untuk setiap prodi dari daftar yang didapat
    for prodi in daftar_prodi_lengkap:
        prodi_id = prodi["id_prodi"]
        nama_prodi = prodi["nama_prodi"]
        nama_fakultas = prodi["nama_fakultas"]
        page = 1
        jurnal_ditemukan_per_prodi = 0
        
        while True:
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{prodi_id}/{page}"
            try:
                r = requests.get(url)
                r.raise_for_status()
                soup = BeautifulSoup(r.content, "html.parser")
                jurnals = soup.select('li[data-cat="#luxury"]')

                if not jurnals:
                    break
                
                jurnal_ditemukan_per_prodi += len(jurnals)

                for jurnal in jurnals:
                    # Proses scraping detail jurnal (tetap sama)
                    jurnal_url = jurnal.select_one('a.gray.button')['href']
                    response = requests.get(jurnal_url)
                    response.raise_for_status()
                    isi = BeautifulSoup(response.content, "html.parser").select_one('div#content_journal')
                    if not isi: continue

                    judul = isi.select_one('a.title').text.strip()
                    penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1].strip()
                    pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1].strip()
                    pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(':')[1].strip()
                    # --- PERUBAHAN PADA EKSTRAKSI ABSTRAK ---
                    abstract_paragraphs = isi.select('p[align="justify"]')
                    
                    # Ambil teks mentah dari abstrak Bahasa Indonesia
                    text_indo_mentah = abstract_paragraphs[0].text if len(abstract_paragraphs) > 0 else "Tidak ada abstrak"
                    # 2. BERSIHKAN TEKS MENTAH
                    abstrak_indonesia = re.sub(r'\s+', ' ', text_indo_mentah).strip()

                    # Ambil teks mentah dari abstrak Bahasa Inggris
                    text_inggris_mentah = abstract_paragraphs[1].text if len(abstract_paragraphs) > 1 else "No abstract available"
                    # 2. BERSIHKAN TEKS MENTAH
                    abstrak_inggris = re.sub(r'\s+', ' ', text_inggris_mentah).strip()
                    # --- AKHIR PERUBAHAN ---

                    # Menambahkan semua data ke dictionary
                    data["nama_fakultas"].append(nama_fakultas)
                    data["id_prodi"].append(prodi_id)
                    data["nama_prodi"].append(nama_prodi)
                    data["penulis"].append(penulis)
                    data["judul"].append(judul)
                    data["pembimbing_pertama"].append(pembimbing_pertama)
                    data["pembimbing_kedua"].append(pembimbing_kedua)
                    data["abstrak"].append(abstrak_indonesia)
                    data["abstrak_inggris"].append(abstrak_inggris)
                
                page += 1
                time.sleep(1)

            except requests.exceptions.RequestException as e:
                print(f"Error pada Prodi {nama_prodi} (ID: {prodi_id}) halaman {page}: {e}")
                break
        
        print(f"✔ Selesai: {nama_fakultas} - {nama_prodi} (ID: {prodi_id}) | Total {jurnal_ditemukan_per_prodi} jurnal.")

    df = pd.DataFrame(data)
    df.to_csv("pta_lengkap_fakultas_prodi.csv", index=False)
    print("\n✅ Proses scraping selesai. Data disimpan ke 'pta_lengkap_fakultas_prodi.csv'")
    
    return df

# Untuk menjalankan seluruh proses scraping
# df_final = scrape_all()
# print(df_final.head())

In [3]:
df_all = scrape_all()



Gagal mengambil daftar fakultas dan prodi: HTTPSConnectionPool(host='pta.trunojoyo.ac.id', port=443): Max retries exceeded with url: /c_search/byfac (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001BDFE616BA0>: Failed to resolve 'pta.trunojoyo.ac.id' ([Errno 11001] getaddrinfo failed)"))
Tidak ada prodi yang bisa di-scrape. Program berhenti.


In [4]:
# === BAGIAN 2: Membaca & Menampilkan SEMUA CSV ===
import pandas as pd

# Atur agar semua baris dan kolom ditampilkan
pd.set_option('display.max_rows', None)   # tampilkan semua baris
pd.set_option('display.max_columns', None)  # tampilkan semua kolom (jika banyak)

df = pd.read_csv("pta_lengkap_fakultas_prodi.csv")

print("=== Semua Data Hasil Crawling ===")
print(df)  # tampilkan semua baris

# Jika di Jupyter Notebook dan ingin tampilan tabel penuh:
df


=== Semua Data Hasil Crawling ===
     nama_fakultas  id_prodi                    nama_prodi  \
0            Hukum         1                    Ilmu Hukum   
1            Hukum         1                    Ilmu Hukum   
2            Hukum         1                    Ilmu Hukum   
3            Hukum         1                    Ilmu Hukum   
4            Hukum         1                    Ilmu Hukum   
5            Hukum         1                    Ilmu Hukum   
6            Hukum         1                    Ilmu Hukum   
7            Hukum         1                    Ilmu Hukum   
8            Hukum         1                    Ilmu Hukum   
9            Hukum         1                    Ilmu Hukum   
10           Hukum         1                    Ilmu Hukum   
11           Hukum         1                    Ilmu Hukum   
12           Hukum         1                    Ilmu Hukum   
13           Hukum         1                    Ilmu Hukum   
14           Hukum         1        

,nama_fakultas,id_prodi,nama_prodi,penulis,judul,pembimbing_pertama,pembimbing_kedua,abstrak,abstrak_inggris
0,Hukum,1,Ilmu Hukum,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",ABSTRAK Implementasi Fungsi Legislasi DPRD Kab...,ABSTRACT Implementation of Legislation Parliam...
1,Hukum,1,Ilmu Hukum,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...
2,Hukum,1,Ilmu Hukum,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,"Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi..."
3,Hukum,1,Ilmu Hukum,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...
4,Hukum,1,Ilmu Hukum,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",NaN,NaN
5,Hukum,1,Ilmu Hukum,Andri Eka Sugiarto,TELAAH KRITIS TENTANG\r\nPEMBERITAAN MEDIA MAS...,"Dr. Deni Setya Bagus Yuherawan, SH., MS","Gatoet Poernomo,SH.M.Hum",ABSTRAK Jaminan akan kemerdekaan dan kebebasan...,ABSTRACT Guarantees of independence and freedo...
6,Hukum,1,Ilmu Hukum,Agung Saiful Anam,PELAKSANAAN PENDAFTARAN TANAH UNTUK PERTAMA KA...,"Dr. Mufarrijul Ikhwan, SH.,M.Hum","Mishbahul Munir, SH., M.Hum",ABSTRAK Pendaftaran tanah untuk pertama kali b...,ABSTRACT First time of Land Registration aims ...
7,Hukum,1,Ilmu Hukum,Adityo Nugroho,TRANSAKSI JUAL BELI MELALUI INTERNET PERSPEKTI...,"Dr. Murni, SH.,M.Hum","Mishbahul Munir, SH., M.Hum",ABSTRAK Salah satu bentuk kemajuan teknologi a...,ABSTRACT One form of technological progress is...
8,Hukum,1,Ilmu Hukum,Yuliano Orgianus Landowero,PELAKSANAAN MORATORIUM PENGIRIMAN TENAGA KERJA...,"Azizah, SH.,M.Hum","Devi Rahayu, SH.,M.Hum",ABSTRAK Pelaksanaan terhadap kebijakan morator...,ABSTRACT The implementation of the moratorium ...
9,Hukum,1,Ilmu Hukum,NUR INDAH FENY SURYANTI,ASPEK PIDANA DALAM JUAL BELI OLI BEKAS KAPAL D...,"BOEDI MUSTIKO, SH., M.Hum","GATOET POERNOMO, SH., M.Hum",Oli bekas kapal merupakan oli sisa yang dihasi...,Ship used oil is residual oil generated from m...
